In [ ]:
import geopandas as gpd
import pandas as pd 
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import warnings
import os


Los archivos a analizar deben de ser almacenados en la carpeta datos, y debe de existir unicamente un archivo shp son sus respectivas correlaciones necesarias.

In [ ]:
# Obtener la ruta de la carpeta "datos"
carpeta = os.path.join(os.getcwd(), 'Datos')

# Buscar el archivo .shp dentro de la carpeta
for archivo in os.listdir(carpeta):
    if archivo.endswith('.shp'):
        # Leer el archivo con GeoPandas
        ruta_archivo = os.path.join(carpeta, archivo)
        red = gpd.read_file(ruta_archivo)
        break

red_df = pd.DataFrame(red)
red_df.to_csv('..\\Programa\\Datos\\nuevo.csv')
# Define una función para extraer las coordenadas de inicio y fin de cada línea
def get_start_end_coords(line):
    start = line.coords[0]
    end = line.coords[-1]
    return pd.Series({'start_lat': start[1], 'start_lon': start[0], 'end_lat': end[1], 'end_lon': end[0]})

# Aplica la función a cada fila del GeoDataFrame y crea cuatro nuevas columnas
red[['start_lat', 'start_lon', 'end_lat', 'end_lon']] = red.geometry.apply(get_start_end_coords)
# Convierte el objeto GeoDataFrame a un DataFrame de pandas
red_df = pd.DataFrame(red)

# Exporta el DataFrame a un archivo csv
red_df.to_csv('..\\Programa\\Datos\\nuevo.csv', index=False)


In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
red.plot(ax=ax, linewidth=0.5)
ax.set_axis_off()

In [ ]:
warnings.filterwarnings('ignore') #Ignora warnings
archivo = pd.read_csv('..\\Programa\\Datos\\nuevo.csv')
archivo


In [ ]:
# Crear los nodos del grafo
G = nx.Graph()

# Obtener la ruta del archivo CSV
ruta_archivo = os.path.join(os.getcwd(), 'Datos', 'nuevo.csv')

# Leer solo las primeras 5 filas del archivo CSV
archivo = pd.read_csv(ruta_archivo, nrows=100)

for i, row in archivo.iterrows():
    origen = (row['start_lon'], row['start_lat'])
    destino = (row['end_lon'], row['end_lat'])
    G.add_node(i, pos=origen)
    G.add_node(i+1, pos=destino)
# Crear los arcos del grafo
for i, row in archivo.iterrows():
    origen = i
    destino = i+1
    peso = row['TV']
    G.add_edge(origen, destino, weight=peso)

# Graficar el grafo en un mapa utilizando mplleaflet
fig, ax = plt.subplots(figsize=(20, 10))
#mplleaflet.display(fig=fig)

nx.draw_networkx(G, pos=nx.get_node_attributes(G, 'pos'), ax=ax, node_size=2, node_color='blue', edge_color='red', with_labels=False, width=1000) 
#mplleaflet.display(fig=fig)

In [ ]:
print(G.edges())
for u, v, w in G.edges.data('weight'):
    print(f"({u}, {v}): {w}")

In [ ]:
# Crear la matriz de incidencia
incidence_matrix = nx.incidence_matrix(G, oriented=False).toarray()

# Crear el DataFrame
nodes = list(G.nodes())
df = pd.DataFrame(data=np.zeros((len(nodes), len(nodes))), index=nodes, columns=nodes)
for i, j, weight in G.edges(data='weight'):
    df.loc[i, j] = weight

# Imprimir la matriz de incidencia y el DataFrame
print(incidence_matrix)
print(df)

In [ ]:
# Calcular los caminos más cortos entre los nodos del subgrafo
shortest_paths = dict(nx.all_pairs_dijkstra_path_length(G, weight='weight'))

# Convertir el diccionario de caminos más cortos en una matriz de distancia
n = len(nodes)
dist_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i, n):
        if i == j:
            dist_matrix[i, j] = 0
        else:
            if nodes[i] in shortest_paths and nodes[j] in shortest_paths[nodes[i]]:
                dist_matrix[i, j] = shortest_paths[nodes[i]][nodes[j]]
                dist_matrix[j, i] = shortest_paths[nodes[i]][nodes[j]]
            else:
                dist_matrix[i, j] = float('inf')
                dist_matrix[j, i] = float('inf')
print(dist_matrix)
# Crear un dataframe de pandas con la matriz
df = pd.DataFrame(dist_matrix, index=nodes, columns=nodes)

# Exportar el dataframe como un archivo CSV
df.to_csv("matriz_AD.csv", index_label='node')


In [ ]:
"""
NO USAR, NO FUNICONA EL CÓDIGO, TIENE UN TIEMPO MUY LARGO EN OPERACIÓN.
"""

# Definir una función para calcular los caminos más cortos entre un nodo origen y todos los demás nodos
def shortest_paths_from_node(G, node):
    return nx.single_source_dijkstra_path_length(G, node, weight='weight')

def process_nodes(G, node_chunk, shortest_paths_shared):
    shortest_paths = {node: shortest_paths_from_node(G, node) for node in node_chunk}
    shortest_paths_shared.update(shortest_paths)

def main():
    G = nx.Graph() # Cargar el grafo
    num_processes = mp.cpu_count() # Determinar la cantidad de procesos óptima para el sistema

    # Dividir los nodos en chunks que contengan un número similar de aristas
    node_chunks = np.array_split(list(G.nodes()), num_processes)

    # Utilizar una estructura de datos más eficiente para almacenar los resultados
    shortest_paths_shared = mp.Manager().dict()

    # Utilizar un enfoque de memoria compartida para permitir que los procesos compartan los datos y los resultados
    process_nodes_partial = functools.partial(process_nodes, G, shortest_paths_shared=shortest_paths_shared)
    
    # Calcular los caminos más cortos desde cada nodo utilizando el Pool de procesos
    with mp.Pool(num_processes) as pool:
        pool.map(process_nodes_partial, node_chunks)

    dist_matrix = pd.DataFrame.from_dict(dict(shortest_paths_shared))

    # Sumar filas y columnas de la matriz
    row_sums = np.sum(dist_matrix.values, axis=1)
    col_sums = np.sum(dist_matrix.values, axis=0)

    # Agregar las sumas como filas y columnas adicionales en la matriz
    dist_matrix['Row Sum'] = row_sums
    dist_matrix.loc['Col Sum'] = np.append(col_sums, np.sum(row_sums))

    print(dist_matrix)

if __name__ == '__main__':
    main()
    